In [1]:
import numpy as np
import scipy as sp
import pandas as pd
from sklearn.linear_model import LinearRegression, LassoCV
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%reload_ext autoreload
%autoreload 2
import itertools
import sys
sys.path.append('/home/roquero/CausalAggregation/Code')
from generateEnvironment import GenerateEnvironment, generate_constraints
from gmm_estimator import SolveProblem
import matplotlib as mpl
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "sans-serif",
    "font.sans-serif": ["Helvetica"]})
np.set_printoptions(suppress=True)
np.set_printoptions(precision=5)
np.set_printoptions(linewidth=1000)

/home/roquero/miniconda3/lib/python3.7/site-packages/matplotlib/__init__.py:886: MatplotlibDeprecationWarning: 
examples.directory is deprecated; in the future, examples will be found relative to the 'datapath' directory.
  "found relative to the 'datapath' directory.".format(key))


In [2]:
n_dim=202
y_index=100
connectivity_e0 = np.zeros((n_dim,n_dim))
z = 2*np.random.binomial(1,0.5, size=n_dim-1)-1
connectivity_e0[np.arange(1,n_dim), np.arange(0,n_dim-1)] = np.random.normal(size=(n_dim-1))*0.5 +1
connectivity_e0[1,0]=0
connectivity_e0[2,0]=1
connectivity_e0[y_index,0]=2
connectivity_e0[n_dim-1,0]=1

x_indices=np.hstack([np.arange(1,y_index),(np.arange(y_index+1,n_dim))])
n_reps=50
alpha=0.05

In [3]:
for n_samples in np.array([100,200,500,1000,5000]):
    coverage =[]
    length = []
    num_selected_indices = []
    for _ in np.arange(n_reps):
        
        solver_obs = SolveProblem(connectivity_e0,x_indices,y_index)
        obs_dataset = solver_obs.generate_intervention(n_samples, {})['dataset']
        
        scaler = StandardScaler()
        obs_dataset = scaler.fit_transform(obs_dataset.T).T
        select_markov = LassoCV(max_iter=5000)
        _ = select_markov.fit(obs_dataset[x_indices,:].T,obs_dataset[y_index,:])
        selected_indices = x_indices[np.where(np.abs(select_markov.coef_)>1e-3)[0]]
        n_selected_indices = len(set(selected_indices).intersection(set([1,2,99,101,200,201])))

        solver = SolveProblem(connectivity_e0,selected_indices,y_index)
        list_dict_interventions = [{i:{'type':'independent'} for i in selected_indices[n_selected_indices//2:]},
                                  {i:{'type':'independent'} for i in selected_indices[:n_selected_indices//2]}] 
        #list_dict_interventions = [{i:{'type':'independent'} for i in selected_indices}] 
        list_environments = [solver.generate_intervention(n_samples, dict_interventions)
                             for dict_interventions in list_dict_interventions]
        n_samples_tot = np.sum([env['dataset'].shape[1] for env in list_environments])
        ### Estimate beta_hat and aCov here
    
        beta_hat, aCov = solver.compute_beta_GMM(list_environments)

        ###
    
        CI = solver.compute_CI(beta_hat, aCov, n_samples_tot, alpha)
        #print(beta_hat, CI, solver.beta)
        for coord in np.arange(n_selected_indices):
            coverage.append((CI[0,coord]<solver.beta[coord])&
                        (CI[1,coord]>solver.beta[coord]))
            length.append(CI[1,coord]-CI[0,coord])
        num_selected_indices.append(n_selected_indices) 
    print('For n_samples {}: Avg coverage: {} pm{}. Avg length: {} pm{}. Avg selected: {} pm{}.'.format(n_samples,np.mean(coverage),
                                                                                                        2*np.std(coverage)/np.sqrt(n_reps),
                                                                                                        np.mean(length),
                                                                                                        2*np.std(length)/np.sqrt(n_reps),
                                                                                                       np.mean(num_selected_indices),
                                                                                                       2*np.std(num_selected_indices)/np.sqrt(n_reps))
         )

For n_samples 100: Avg coverage: 0.9926470588235294 pm0.02416423047816909. Avg length: 531.0018707904289 pm1303.0086065765104. Avg selected: 2.72 pm0.2594763958436297.
For n_samples 200: Avg coverage: 0.959731543624161 pm0.0556035306770684. Avg length: 1.0956281225463527 pm0.3371481638748519. Avg selected: 2.98 pm0.2560624923724675.
For n_samples 500: Avg coverage: 0.9679144385026738 pm0.049844621166763824. Avg length: 0.5496504173605108 pm0.13788880923586438. Avg selected: 3.74 pm0.16790473489452284.
For n_samples 1000: Avg coverage: 0.9617021276595744 pm0.054281595639496076. Avg length: 0.3617865117688198 pm0.07900523547691322. Avg selected: 4.7 pm0.26683328128252665.
For n_samples 5000: Avg coverage: 0.939622641509434 pm0.06736879578693253. Avg length: 0.14255487613604517 pm0.02862221056335635. Avg selected: 5.3 pm0.12961481396815722.


In [4]:
for n_samples in np.array([100,200,500,1000,5000]):
    coverage=[]
    length=[]
    num_selected_indices = []
    for _ in np.arange(n_reps):
        
        solver_obs = SolveProblem(connectivity_e0,x_indices,y_index)
        obs_dataset = solver_obs.generate_intervention(n_samples, {})['dataset']
        
        scaler = StandardScaler()
        obs_dataset = scaler.fit_transform(obs_dataset.T).T
        select_markov = LassoCV(max_iter=5000)
        _ = select_markov.fit(obs_dataset[x_indices,:].T,obs_dataset[y_index,:])
        selected_indices = x_indices[np.where(np.abs(select_markov.coef_)>1e-3)[0]]
        n_selected_indices = len(selected_indices)
        
        solver = SolveProblem(connectivity_e0,selected_indices,y_index)
        list_dict_interventions = [{i:{'type':'independent'} for i in selected_indices[n_selected_indices//2:]},
                                  {i:{'type':'independent'} for i in selected_indices[:n_selected_indices//2]}]       
        list_environments = [solver.generate_intervention(n_samples, dict_interventions)
                             for dict_interventions in list_dict_interventions]
        n_samples_tot = np.sum([env['dataset'].shape[1] for env in list_environments])
        ### Estimate beta_hat and aCov here
    
        beta_hat, aCov = solver.compute_pooled_beta_OLS(list_environments)
  
        ###
    
        CI = solver.compute_CI(beta_hat, aCov, n_samples_tot, alpha)
        
        for coord in np.arange(n_selected_indices):
            coverage.append((CI[0,coord]<solver.beta[coord])&
                        (CI[1,coord]>solver.beta[coord]))
            length.append(CI[1,coord]-CI[0,coord])
            
    print('For n_samples {}: Avg coverage: {} pm{}. Avg length: {} pm{}.'.format(n_samples,np.mean(coverage),2*np.std(coverage)/np.sqrt(n_reps),np.mean(length),2*np.std(length)/np.sqrt(n_reps))
         )

For n_samples 100: Avg coverage: 0.6702355460385439 pm0.1329721350855009. Avg length: 0.36488433859184 pm0.03318998269387847.
For n_samples 200: Avg coverage: 0.6195028680688337 pm0.13732270446603442. Avg length: 0.24285437615444658 pm0.01942379482814677.
For n_samples 500: Avg coverage: 0.5255198487712666 pm0.14123703121170217. Avg length: 0.1489905193906185 pm0.01242040478341625.
For n_samples 1000: Avg coverage: 0.5473537604456824 pm0.1407856871621978. Avg length: 0.10785139062599561 pm0.0085728260236467.
For n_samples 5000: Avg coverage: 0.42543171114599687 pm0.13983978555400775. Avg length: 0.04897066893444497 pm0.003845301282116153.
